In [ ]:
#| default_exp routes.job

In [ ]:
#| exporti
from typing import Union
import httpx
from pprint import pprint

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda


In [ ]:
#| export

# get RemoteDomostats job names
async def get_jobs(auth: dmda.DomoFullAuth,
                   application_id: str,
                   debug_api: bool = False,
                   session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None)-> rgd.ResponseGetData:
    
    offset_params = {
            'offset': 'offset',
            'limit': 'limit'}

    url = f'https://{auth.domo_instance}.domo.com/api/executor/v2/applications/{application_id}/jobs'

    if debug_api:
        print(url)

    def arr_fn(res) -> list[dict]:
        return res.response.get('jobs')


    # deprecated from looper 
    # def alter_maximum_fn(res):
    #     return res.response.get('totalResults')

    return await gd.looper(auth=auth,
                           method='GET',
                           url=url,
                           arr_fn=arr_fn,
                           loop_until_end=True,
                           offset_params=offset_params,
                           session=session,
                           debug_api=debug_api)


# create the new RemoteDomostats job
async def add_job(auth: dmda.DomoFullAuth,
                  body: dict,
                  application_id: str,
                  session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None,
                  debug_api: bool = False
                  )-> rgd.ResponseGetData:

    url = f'https://{auth.domo_instance}.domo.com/api/executor/v1/applications/{application_id}/jobs'

    if debug_api:
        print(url)

    return await gd.get_data(
        auth=auth,
        url=url,
        method='POST',
        body=body,
        debug_api=debug_api,
        session=session
    )


def generate_body_remote_domostats(target_instance: str,
                                   report_dict: dict,
                                   output_dataset_id: str,
                                   account_id: str,
                                   schedule_ls: list,
                                   execution_timeout: int = 1440,
                                   debug_api: bool = False):

    instance_url = f"{target_instance}.domo.com"

    body = {
        "jobName": instance_url,
        "jobDescription": f'Get Remote stat from {instance_url}',
        "executionTimeout": execution_timeout,
        "executionPayload": {
            "remoteInstance": instance_url,
            "policies": report_dict,
            "metricsDatasetId": output_dataset_id},
        "accounts": [account_id],
        "executionClass": "com.domo.executor.subscriberstats.SubscriberStatsExecutor",
        "resources": {
            "requests": {"memory": "256M"},
            "limits": {"memory": "256M"}},
        "triggers": schedule_ls
    }

    if debug_api:
        pprint(body)

    return body


def generate_body_watchdog_generic(job_name: str,
                                        notify_user_ids_ls: list,
                                        notify_group_ids_ls: list,
                                        notify_emails_ls: list,
                                        entity_ids_ls: list,
                                        entity_type : str,
                                        metric_dataset_id: str,
                                        schedule_ls: list,
                                        job_type : str,
                                        execution_timeout: int = 1440,
                                        debug_api: bool = False):

    body = {
        "jobName": job_name,
        "jobDescription": f'Watchdog for {job_name}',
        "executionTimeout": execution_timeout,
        "accounts": [],
        "executionPayload": {
            "notifyUserIds": notify_user_ids_ls or [],
            "notifyGroupIds": notify_group_ids_ls or [],
            "notifyEmailAddresses": notify_emails_ls or [],
        "watcherParameters": {
          "entityIds": entity_ids_ls,
          "type": job_type,
          "entityType": entity_type
            },
        "metricsDatasetId": metric_dataset_id
          },
        "resources": {
            "requests": {"memory": "256Mi"},
            "limits": {"memory": "256Mi"}
        },
        "triggers": schedule_ls
    }

    if debug_api:
        pprint(body)

    return body

# update the job
async def update_job(auth: dmda.DomoFullAuth,
                  body: dict,
                  job_id :str,
                  application_id: str,
                  session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None,
                  debug_api: bool = False
                  )-> rgd.ResponseGetData:

    url = f'https://{auth.domo_instance}.domo.com/api/executor/v1/applications/{application_id}/jobs/{job_id}'

    if debug_api:
        print(url)

    return await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        body=body,
        debug_api=debug_api,
        session=session
    )

#update trigger

async def update_job_trigger(auth: dmda.DomoFullAuth,
                  body: dict,
                  job_id :str,
                  trigger_id : str,  
                  application_id: str,
                  session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None,
                  debug_api: bool = False
                  )-> rgd.ResponseGetData:

    url = f'https://{auth.domo_instance}.domo.com/api/executor/v1/applications/{application_id}/jobs/{job_id}/triggers/{trigger_id}'

    if debug_api:
        print(url)

    return await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        body=body,
        debug_api=debug_api,
        session=session
    )

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)
application_id = "50e7230f-d2f2-42e2-a208-d94c8ae9f64c"

res = await get_jobs(auth = token_auth, application_id = application_id)
print (res)

ResponseGetData(status=200, response=[], is_success=True)
